In [1]:
import numpy as np
import pandas as pd
import sys
import ipywidgets as widgets
from IPython.display import display, clear_output
from IPython.display import Image
import time
import matplotlib 
import matplotlib.pyplot as plot

In [2]:
sys.path.append('/usr/local/Cellar/graph-tool/2.53/lib/python3.11/site-packages')

In [3]:
import graph_tool
from graph_tool.all import *

s = pd.read_csv("toy_example_s_matrix.csv")

In [4]:
s = pd.read_csv("whole_cell_s_matrix (1).csv")

In [5]:
s

,Unnamed: 0,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),1PFRUCTPHOSN-RXN,1TRANSKETO-RXN (reverse),2-DEHYDROPANTOATE-REDUCT-RXN (reverse),2-ISOPROPYLMALATESYN-RXN,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,...,ISOCHORSYN-RXN__MENF-CPLX,RXN-9535__FABB-CPLX,RXN0-313__CPLX0-201 (reverse),RXN0-384__H2NEOPTERINP3PYROPHOSPHOHYDRO-MONOMER,RXN0-5186__G7408-MONOMER,SERINE-O-ACETTRAN-RXN__CPLX0-237,SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX,UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//CPD-9646/Pi/PROTON.64.__PGPPHOSPHAB-MONOMER,URPHOS-RXN__URPHOS-CPLX,maintenance_reaction
0,NAD[c],1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NADH[c],-1,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PROTON[c],-1,-1,1,0,-1,1,-1,1,1,...,0,-1,0,1,0,0,0,1,0,1
3,GLUCONATE[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NADP[c],0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,R-3-hydroxymyristoyl-ACPs[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
485,Adenylated-ThiS-Proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
486,ThiS-CoASH-proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,Thi-S[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
flow = pd.read_csv("whole_cell_flow_data.csv")

In [7]:
# Make edges
edges = set({})
vertexes = []
reactions = set({})
metabolites = set({})
for column in s:
    if column != "Unnamed: 0":
        vertexes.append(column)
        reactions.add(column)
    for i, item in s[column].items():
        if column == "Unnamed: 0":
            vertexes.append(s["Unnamed: 0"][i])
            metabolites.add(s["Unnamed: 0"][i])
        elif item < 0:
            edge = (s["Unnamed: 0"][i], column)
            edges.add(edge)
        elif item > 0:
            edge = (column, s["Unnamed: 0"][i])
            edges.add(edge)

In [8]:
vertex_dict = {name:i for i, name in enumerate(vertexes)}

In [9]:
new_edges = []
for i, edge in enumerate(edges):
    new_edges.append((vertex_dict[edge[0]], vertex_dict[edge[1]]))
edges = new_edges

In [10]:
g = Graph()
g.add_edge_list(edges)
degrees = g.degree_property_map("out")

In [11]:
names = g.new_vertex_property("string")
colors = g.new_vertex_property("string")
shapes = g.new_vertex_property("string")

for i, name in enumerate(vertexes):
    names[g.vertex(i)] = name
    
    if name in reactions:
        shapes[g.vertex(i)] = "square"
        colors[g.vertex(i)] = "blue"
    elif name in metabolites:
        shapes[g.vertex(i)] = "circle"
        colors[g.vertex(i)] = "red"

g.vertex_properties["name"] = names
g.vertex_properties["shape"] = shapes
g.vertex_properties["color"] = colors

In [12]:
pos = graph_tool.draw.sfdp_layout(g)
graph_draw(g, pos=pos, vertex_text=g.vertex_properties["name"],
           vertex_shape=g.vertex_properties["shape"],
           vertex_fill_color=g.vertex_properties["color"],
           vertex_font_size=5, vertex_text_position=3.14,
           vertex_text_offset=[1.0,0], output="named_vertices.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x167071cd0, at 0x16701ec50>

In [13]:
max_degree_vertices = sorted(range(g.num_vertices()), key=lambda i: degrees[i], reverse=True)[:100]

mask = np.ones(g.num_vertices())
mask[max_degree_vertices] = False
gf = graph_tool.GraphView(g, vfilt=mask)

In [125]:
pos = graph_tool.draw.sfdp_layout(gf)
graph_draw(gf, pos=pos, vertex_shape=g.vertex_properties["shape"],
           vertex_fill_color=g.vertex_properties["color"],
           vertex_font_size=5, vertex_text_position=3.14,
           vertex_text_offset=[1.0,0], output="named_vertices.pdf")

ValueError: zero-size array to reduction operation minimum which has no identity

In [ ]:
flow

In [ ]:
# Set initial values for width and height
initial_width = 400
initial_height = 300

# Create sliders for width and height
width_slider = widgets.IntSlider(value=initial_width, min=100, max=800, step=50, description='Width:')
height_slider = widgets.IntSlider(value=initial_height, min=100, max=600, step=50, description='Height:')

# Define the graph drawing function
def draw_graph(width, height):
    pos = graph_tool.draw.sfdp_layout(gf)
    graph_draw(gf, pos=pos, vertex_shape=g.vertex_properties["shape"],
               vertex_fill_color=g.vertex_properties["color"],
               vertex_font_size=5, vertex_text_position=3.14,
               vertex_text_offset=[1.0,0], output="test.png")
    display(Image(filename="test.png"))

# Define the callback function for slider changes
def handle_slider_change(change):
    width = width_slider.value
    height = height_slider.value
    time.sleep(1)
    clear_output(wait=True) 
    display(widgets.VBox([width_slider, height_slider]))
    draw_graph(width, height)

# Attach the callback function to the slider value change events
width_slider.observe(handle_slider_change, names='value')
height_slider.observe(handle_slider_change, names='value')

# Display the sliders
display(width_slider)
display(height_slider)

# Initial graph drawing
draw_graph(initial_width, initial_height)

## Stoichmatrix to 1s

In [6]:
s

,Unnamed: 0,1.5.1.20-RXN-5-METHYL-THF/NAD//METHYLENE-THF/NADH/PROTON.44. (reverse),1.5.1.20-RXN-CPD-1302/NAD//CPD-12996/NADH/PROTON.36. (reverse),1PFRUCTPHOSN-RXN,1TRANSKETO-RXN (reverse),2-DEHYDROPANTOATE-REDUCT-RXN (reverse),2-ISOPROPYLMALATESYN-RXN,2-OCTAPRENYL-6-METHOXYPHENOL-HYDROX-RXN,2-OCTAPRENYL-6-OHPHENOL-METHY-RXN,2-OCTAPRENYL-METHOXY-BENZOQ-METH-RXN,...,ISOCHORSYN-RXN__MENF-CPLX,RXN-9535__FABB-CPLX,RXN0-313__CPLX0-201 (reverse),RXN0-384__H2NEOPTERINP3PYROPHOSPHOHYDRO-MONOMER,RXN0-5186__G7408-MONOMER,SERINE-O-ACETTRAN-RXN__CPLX0-237,SERINE-O-ACETTRAN-RXN__CYSSYNMULTI-CPLX,UNDECAPRENYL-DIPHOSPHATASE-RXN[CCO-CYTOSOL]-UNDECAPRENYL-DIPHOSPHATE/WATER//CPD-9646/Pi/PROTON.64.__PGPPHOSPHAB-MONOMER,URPHOS-RXN__URPHOS-CPLX,maintenance_reaction
0,NAD[c],1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NADH[c],-1,-1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,PROTON[c],-1,-1,1,0,-1,1,-1,1,1,...,0,-1,0,1,0,0,0,1,0,1
3,GLUCONATE[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NADP[c],0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484,R-3-hydroxymyristoyl-ACPs[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
485,Adenylated-ThiS-Proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
486,ThiS-CoASH-proteins[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
487,Thi-S[c],0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
stoich = s.values[:, 1:]
stoich[stoich != 0] = 1
cooc = stoich.dot(stoich.T)
cooc

array([[38, 36, 33, ..., 0, 0, 0],
       [36, 36, 31, ..., 0, 0, 0],
       [33, 31, 236, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 2, 1, 0],
       [0, 0, 1, ..., 1, 2, 0],
       [0, 0, 0, ..., 0, 0, 1]], dtype=object)

In [8]:
n, _ = cooc.shape

In [9]:
ones = np.tril(np.ones((n, n))) - np.eye(n)

In [10]:
ones

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 1., 0., 0.],
       [1., 1., 1., ..., 1., 1., 0.]])

In [11]:
test = cooc * ones
test

array([[0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [36.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [33.0, 31.0, 0.0, ..., 0.0, 0.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [0.0, 0.0, 1.0, ..., 1.0, 0.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [12]:
stoich_dict = {t[0]:t[1] for t in list(s["Unnamed: 0"].items())}

In [13]:
test_flat = test.flatten()

In [14]:
test_flat

array([0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0], dtype=object)

In [15]:
test_tups = [(val, (stoich_dict[i // n], stoich_dict[i % n])) for i, val in enumerate(test_flat)]

In [16]:
stoich_dict

{0: 'NAD[c]',
 1: 'NADH[c]',
 2: 'PROTON[c]',
 3: 'GLUCONATE[c]',
 4: 'NADP[c]',
 5: 'NADPH[c]',
 6: 'MAL[c]',
 7: 'CARBON-DIOXIDE[c]',
 8: 'PYRUVATE[c]',
 9: 'Red-Thioredoxin[c]',
 10: 'Ox-Thioredoxin[c]',
 11: 'WATER[c]',
 12: 'OXYGEN-MOLECULE[c]',
 13: '5-METHYL-THF[c]',
 14: 'METHYLENE-THF[c]',
 15: 'CPD-1302[c]',
 16: 'CPD-12996[c]',
 17: 'GLUTATHIONE[c]',
 18: 'OXIDIZED-GLUTATHIONE[c]',
 19: 'CYS[c]',
 20: 'ATP[c]',
 21: 'FRU1P[c]',
 22: 'ADP[c]',
 23: 'FRUCTOSE-16-DIPHOSPHATE[c]',
 24: 'D-SEDOHEPTULOSE-7-P[c]',
 25: 'GAP[c]',
 26: 'RIBOSE-5P[c]',
 27: 'XYLULOSE-5-PHOSPHATE[c]',
 28: 'L-PANTOATE[c]',
 29: '2-DEHYDROPANTOATE[c]',
 30: '2-KETO-ISOVALERATE[c]',
 31: 'ACETYL-COA[c]',
 32: '3-CARBOXY-3-HYDROXY-ISOCAPROATE[c]',
 33: 'CO-A[c]',
 34: 'OXALACETIC_ACID[c]',
 35: 'PROPIONYL-COA[c]',
 36: '2-OCTAPRENYL-6-METHOXYPHENOL[c]',
 37: 'OCTAPRENYL-METHOXY-BENZOQUINONE[c]',
 38: 'S-ADENOSYLMETHIONINE[c]',
 39: '2-OCTAPRENYL-6-HYDROXYPHENOL[c]',
 40: 'ADENOSYL-HOMO-CYS[c]',
 41: 'OCTA

In [17]:
sorted_tups = sorted(test_tups, key=lambda x:x[0], reverse=True)

In [18]:
sorted_tups

[(57.0, ('ADP[c]', 'ATP[c]')),
 (56.0, ('ATP[c]', 'PROTON[c]')),
 (51.0, ('WATER[c]', 'PROTON[c]')),
 (42.0, ('CARBON-DIOXIDE[c]', 'PROTON[c]')),
 (42.0, ('ADP[c]', 'PROTON[c]')),
 (36.0, ('NADH[c]', 'NAD[c]')),
 (35.0, ('Pi[c]', 'PROTON[c]')),
 (34.0, ('Pi[c]', 'WATER[c]')),
 (33.0, ('PROTON[c]', 'NAD[c]')),
 (32.0, ('NADPH[c]', 'NADP[c]')),
 (31.0, ('PROTON[c]', 'NADH[c]')),
 (30.0, ('NADP[c]', 'PROTON[c]')),
 (29.0, ('NADPH[c]', 'PROTON[c]')),
 (27.0, ('PPI[c]', 'PROTON[c]')),
 (23.0, ('Pi[c]', 'ATP[c]')),
 (22.0, ('Pi[c]', 'ADP[c]')),
 (18.0, ('PPI[c]', 'ATP[c]')),
 (14.0, ('GLT[c]', '2-KETOGLUTARATE[c]')),
 (13.0, ('CO-A[c]', 'ACETYL-COA[c]')),
 (13.0, ('AMP[c]', 'ATP[c]')),
 (11.0, ('PYRUVATE[c]', 'PROTON[c]')),
 (11.0, ('AMP[c]', 'PROTON[c]')),
 (11.0, ('ACP[c]', 'PROTON[c]')),
 (10.0, ('CO-A[c]', 'PROTON[c]')),
 (10.0, ('AMP[c]', 'PPI[c]')),
 (10.0, ('PROTON[p]', 'PROTON[c]')),
 (10.0, ('AMMONIUM[c]', 'PROTON[c]')),
 (10.0, ('MALONYL-ACP[c]', 'PROTON[c]')),
 (10.0, ('MALONYL-AC

In [156]:
cull = [tup for tup in sorted_tups if tup[0] > 2]

In [157]:
cull

[(57.0, ('ADP[c]', 'ATP[c]')),
 (56.0, ('ATP[c]', 'PROTON[c]')),
 (51.0, ('WATER[c]', 'PROTON[c]')),
 (42.0, ('CARBON-DIOXIDE[c]', 'PROTON[c]')),
 (42.0, ('ADP[c]', 'PROTON[c]')),
 (36.0, ('NADH[c]', 'NAD[c]')),
 (35.0, ('Pi[c]', 'PROTON[c]')),
 (34.0, ('Pi[c]', 'WATER[c]')),
 (33.0, ('PROTON[c]', 'NAD[c]')),
 (32.0, ('NADPH[c]', 'NADP[c]')),
 (31.0, ('PROTON[c]', 'NADH[c]')),
 (30.0, ('NADP[c]', 'PROTON[c]')),
 (29.0, ('NADPH[c]', 'PROTON[c]')),
 (27.0, ('PPI[c]', 'PROTON[c]')),
 (23.0, ('Pi[c]', 'ATP[c]')),
 (22.0, ('Pi[c]', 'ADP[c]')),
 (18.0, ('PPI[c]', 'ATP[c]')),
 (14.0, ('GLT[c]', '2-KETOGLUTARATE[c]')),
 (13.0, ('CO-A[c]', 'ACETYL-COA[c]')),
 (13.0, ('AMP[c]', 'ATP[c]')),
 (11.0, ('PYRUVATE[c]', 'PROTON[c]')),
 (11.0, ('AMP[c]', 'PROTON[c]')),
 (11.0, ('ACP[c]', 'PROTON[c]')),
 (10.0, ('CO-A[c]', 'PROTON[c]')),
 (10.0, ('AMP[c]', 'PPI[c]')),
 (10.0, ('PROTON[p]', 'PROTON[c]')),
 (10.0, ('AMMONIUM[c]', 'PROTON[c]')),
 (10.0, ('MALONYL-ACP[c]', 'PROTON[c]')),
 (10.0, ('MALONYL-AC

In [158]:
cull = [item[1] for item in cull]

In [159]:
cull

[('ADP[c]', 'ATP[c]'),
 ('ATP[c]', 'PROTON[c]'),
 ('WATER[c]', 'PROTON[c]'),
 ('CARBON-DIOXIDE[c]', 'PROTON[c]'),
 ('ADP[c]', 'PROTON[c]'),
 ('NADH[c]', 'NAD[c]'),
 ('Pi[c]', 'PROTON[c]'),
 ('Pi[c]', 'WATER[c]'),
 ('PROTON[c]', 'NAD[c]'),
 ('NADPH[c]', 'NADP[c]'),
 ('PROTON[c]', 'NADH[c]'),
 ('NADP[c]', 'PROTON[c]'),
 ('NADPH[c]', 'PROTON[c]'),
 ('PPI[c]', 'PROTON[c]'),
 ('Pi[c]', 'ATP[c]'),
 ('Pi[c]', 'ADP[c]'),
 ('PPI[c]', 'ATP[c]'),
 ('GLT[c]', '2-KETOGLUTARATE[c]'),
 ('CO-A[c]', 'ACETYL-COA[c]'),
 ('AMP[c]', 'ATP[c]'),
 ('PYRUVATE[c]', 'PROTON[c]'),
 ('AMP[c]', 'PROTON[c]'),
 ('ACP[c]', 'PROTON[c]'),
 ('CO-A[c]', 'PROTON[c]'),
 ('AMP[c]', 'PPI[c]'),
 ('PROTON[p]', 'PROTON[c]'),
 ('AMMONIUM[c]', 'PROTON[c]'),
 ('MALONYL-ACP[c]', 'PROTON[c]'),
 ('MALONYL-ACP[c]', 'CARBON-DIOXIDE[c]'),
 ('ACP[c]', 'CARBON-DIOXIDE[c]'),
 ('ACP[c]', 'MALONYL-ACP[c]'),
 ('PYRUVATE[c]', 'CARBON-DIOXIDE[c]'),
 ('ACETYL-COA[c]', 'PROTON[c]'),
 ('S-ADENOSYLMETHIONINE[c]', 'PROTON[c]'),
 ('GLT[c]', 'PROTON[c]

In [160]:
# Make edges
edges = set({})
vertexes = []
reactions = set({})
metabolites = set({})
metabolite_dict = {}

for column in s:
    if column != "Unnamed: 0":
        # vertexes.append(column)
        reactions.add(column)
    for i, item in s[column].items():
        if column == "Unnamed: 0":
            # vertexes.append(s["Unnamed: 0"][i])
            metabolites.add(s["Unnamed: 0"][i])
        else:
            if column not in metabolite_dict:
                metabolite_dict[column] = set({})
            if item != 0:
                metabolite_dict[column].add(s["Unnamed: 0"][i])
            if item < 0:
                edge = (s["Unnamed: 0"][i], column)
                edges.add(edge)
            elif item > 0:
                edge = (column, s["Unnamed: 0"][i])
                edges.add(edge)

In [161]:
for reaction in reactions:
    molecules = metabolite_dict[reaction]
    
    for pair in cull:
        if pair[0] in molecules and pair[1] in molecules:
            if (pair[0], reaction) in edges:
                edges.remove((pair[0], reaction))
            elif (reaction, pair[0]) in edges:
                edges.remove((reaction, pair[0]))
            
            if (pair[1], reaction) in edges:
                edges.remove((pair[1], reaction))
            elif (reaction, pair[1]) in edges:
                edges.remove((reaction, pair[1]))

In [162]:
vertexes = set({})
for edge in edges:
    for metabol in edge:
        vertexes.add(metabol)

In [163]:
vertex_dict = {name:i for i, name in enumerate(vertexes)}

In [164]:
new_edges = []
for edge in edges:
    new_edges.append((vertex_dict[edge[0]], vertex_dict[edge[1]]))
edges = new_edges

In [165]:
g = Graph()
g.add_edge_list(edges)
degrees = g.degree_property_map("out")

In [166]:
names = g.new_vertex_property("string")
colors = g.new_vertex_property("string")
shapes = g.new_vertex_property("string")

for i, name in enumerate(vertexes):
    names[g.vertex(i)] = name
    
    if name in reactions:
        shapes[g.vertex(i)] = "square"
        colors[g.vertex(i)] = "blue"
    elif name in metabolites:
        shapes[g.vertex(i)] = "circle"
        colors[g.vertex(i)] = "red"

g.vertex_properties["name"] = names
g.vertex_properties["shape"] = shapes
g.vertex_properties["color"] = colors

In [167]:
pos = graph_tool.draw.sfdp_layout(g)
graph_draw(g, pos=pos, #vertex_text=g.vertex_properties["name"],
           vertex_shape=g.vertex_properties["shape"],
           vertex_fill_color=g.vertex_properties["color"],
           vertex_font_size=5, vertex_text_position=3.14,
           vertex_text_offset=[1.0,0], output="culled_vertices.pdf")

<VertexPropertyMap object with value type 'vector<double>', for Graph 0x1922cff10, at 0x16828bf90>